In [ ]:
import jax.random as jrand

import genjax
from genjax import default_seed, seed

p = 0.5


@genjax.gen
def f():
    flip = genjax.flip(p) @ "flip"
    return (
        genjax.normal(0.0, 1.0).or_else(genjax.normal(2.0, 1.0))(flip, (), ()) @ "value"
    )


args = ()
tr = seed(f.simulate)(jrand.key(1), args)
score, ret = default_seed(f.assess)(tr.get_choices(), args)
choices = tr.get_choices()
choices

In [ ]:
f.assess(choices, args)